In [45]:
from openai import OpenAI
import os
import json
from datetime import datetime
from pathlib import Path
from pypdf import PdfReader

# Toma la API key desde la variable de entorno
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MEMORY_FILE = "memoria.json"
PROFILE_FILE = "perfil_usuario.json"  
# Historial de la conversación proactiva (solo texto)
CONVERSACION_PROACTIVA = []



## Perfil de la persona (carga inicial + actualización segun comentarios de la misma persona)

In [ ]:
def perfil_por_defecto():
    return {
        "nombre": None,
        "edad": None,
        "fecha_nacimiento": None,
        "genero": None,
        "condiciones_salud": [],
        "medicamentos_cronicos": [],
        "preferencias": {
            "tratamiento": None,          # "tuteo" / "usted"
            "temas_favoritos": [],        # ej: ["nietos", "jardinería"]
            "tono_respuesta": None        # ej: "cálido y paciente"
        },
        "familia": []                     # lista de {"nombre":..,"relacion":..}
    }

def cargar_perfil():
    """Carga el perfil desde disco o devuelve uno por defecto si no existe."""
    if not os.path.exists(PROFILE_FILE):
        return perfil_por_defecto()
    with open(PROFILE_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

def guardar_perfil(perfil: dict):
    """Guarda el perfil completo en disco."""
    with open(PROFILE_FILE, "w", encoding="utf-8") as f:
        json.dump(perfil, f, ensure_ascii=False, indent=2)


In [ ]:
perfil = cargar_perfil()
#perfil["nombre"] = "María"
#guardar_perfil(perfil)

perfil

In [ ]:
def extraer_info_perfil_desde_texto(texto: str) -> dict:
    """
    Usa gpt-4o-mini para ver si el texto contiene información del perfil del usuario.
    Devuelve un dict del tipo:
    {
      "contiene_info_perfil": true/false,
      "campos": {
         ... solo los campos que hay que actualizar ...
      }
    }
    """

    system_msg = """
Eres un asistente que extrae INFORMACIÓN DE PERFIL de una persona mayor,
a partir de lo que dice en voz alta.

El perfil tiene esta estructura:

{
  "nombre": string o null,
  "edad": number o null,
  "fecha_nacimiento": string (AAAA-MM-DD) o null,
  "genero": string o null,
  "condiciones_salud": [string, ...],
  "medicamentos_cronicos": [string, ...],
  "preferencias": {
    "tratamiento": string o null,        // "tuteo" o "usted" u otra cosa textual
    "temas_favoritos": [string, ...],
    "tono_respuesta": string o null
  },
  "familia": [
    {"nombre": string, "relacion": string}
  ]
}

Tu tarea:

1. Decidir si el texto contiene información relevante para el perfil.
   Ejemplos de información relevante:
   - nombre, edad, fecha de nacimiento
   - "tengo diabetes", "soy hipertenso", "soy alérgica a..."
   - "tomo metformina todos los días"
   - "me gusta que me hablen de usted"
   - "me gusta hablar de mis nietos"
   - "mi hijo Juan vive en tal parte"

2. Si NO hay información de perfil → responde:
   {"contiene_info_perfil": false, "campos": {}}

3. Si SÍ hay información de perfil → responde:
   {
     "contiene_info_perfil": true,
     "campos": {
        ... SOLO los campos que se pueden inferir del texto ...
     }
   }

Reglas importantes:
- NO inventes datos que no estén en el texto.
- Si mencionan varias condiciones de salud, usa una lista en "condiciones_salud".
- Si hay nuevos medicamentos crónicos, ponlos en "medicamentos_cronicos".
- En "familia", solo agrega personas si se menciona claramente la relación (hijo, hija, nieto, etc.).
- Si no estás seguro de un campo, omítelo.
- Devuelve SIEMPRE un JSON válido y NADA más.
"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": texto},
        ],
    )

    raw = completion.choices[0].message.content

    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        print("No se pudo parsear JSON al extraer perfil. Respuesta cruda:")
        print(raw)
        raise

    # normalizar
    if "contiene_info_perfil" not in data:
        data["contiene_info_perfil"] = False
    if "campos" not in data:
        data["campos"] = {}

    return data

def merge_preferencias(pref_actual: dict, pref_nuevas: dict) -> dict:
    """Fusiona el sub-dict de preferencias."""
    if pref_actual is None:
        pref_actual = {"tratamiento": None, "temas_favoritos": [], "tono_respuesta": None}

    pref = pref_actual.copy()

    if "tratamiento" in pref_nuevas and pref_nuevas["tratamiento"] is not None:
        pref["tratamiento"] = pref_nuevas["tratamiento"]

    if "tono_respuesta" in pref_nuevas and pref_nuevas["tono_respuesta"] is not None:
        pref["tono_respuesta"] = pref_nuevas["tono_respuesta"]

    if "temas_favoritos" in pref_nuevas:
        # agregamos sin duplicar
        actuales = set(pref.get("temas_favoritos") or [])
        nuevos = set(pref_nuevas["temas_favoritos"] or [])
        pref["temas_favoritos"] = list(actuales.union(nuevos))

    return pref

def merge_familia(familia_actual: list, familia_nueva: list) -> list:
    """
    Fusiona listas de familia sin duplicar.
    Considera duplicado si nombre + relacion coinciden.
    """
    if familia_actual is None:
        familia_actual = []
    existentes = {(m.get("nombre"), m.get("relacion")) for m in familia_actual}
    resultado = list(familia_actual)
    for miembro in familia_nueva or []:
        key = (miembro.get("nombre"), miembro.get("relacion"))
        if key not in existentes:
            resultado.append(miembro)
            existentes.add(key)
    return resultado

def actualizar_perfil_con_campos(perfil: dict, campos: dict) -> dict:
    """
    Actualiza el perfil con los campos extraídos del LLM.
    Solo pisa / agrega lo que venga en 'campos'.
    """
    p = perfil.copy()

    # Campos simples
    for k in ["nombre", "edad", "fecha_nacimiento", "genero"]:
        if k in campos and campos[k] is not None:
            p[k] = campos[k]

    # Listas: condiciones_salud, medicamentos_cronicos
    for k in ["condiciones_salud", "medicamentos_cronicos"]:
        if k in campos:
            actuales = set(p.get(k) or [])
            nuevos = set(campos[k] or [])
            p[k] = list(actuales.union(nuevos))

    # Preferencias (sub-dict)
    if "preferencias" in campos:
        p["preferencias"] = merge_preferencias(
            p.get("preferencias"),
            campos["preferencias"]
        )

    # Familia (lista de personas)
    if "familia" in campos:
        p["familia"] = merge_familia(
            p.get("familia"),
            campos["familia"]
        )

    return p


In [ ]:
def procesar_texto_para_perfil(texto: str) -> dict:
    """
    Dado un texto (por ej. transcripción de audio), detecta si hay info de perfil,
    actualiza el perfil en disco y devuelve el perfil actualizado.
    """
    perfil = cargar_perfil()
    extraido = extraer_info_perfil_desde_texto(texto)

    if not extraido.get("contiene_info_perfil"):
        print("No se detectó información de perfil en este texto.")
        return perfil

    campos = extraido.get("campos", {})
    perfil_actualizado = actualizar_perfil_con_campos(perfil, campos)
    guardar_perfil(perfil_actualizado)

    print("Perfil actualizado con:")
    print(json.dumps(campos, ensure_ascii=False, indent=2))

    return perfil_actualizado

In [ ]:
texto_demo = "Me llamo Elvira Soto, tengo 79 años, soy diabética y me gusta que me hablen de tú."
perfil_act = procesar_texto_para_perfil(texto_demo)
perfil_act


In [ ]:


def cargar_perfil_simple():
    """
    Si ya tienes otra función cargar_perfil(), puedes usarla directamente.
    Esta es por si no la tienes centralizada.
    """
    if os.path.exists(PROFILE_FILE):
        with open(PROFILE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}  # perfil vacío si aún no hay info


def construir_contexto_perfil(perfil: dict) -> str:
    """
    Convierte el perfil en un texto corto para el prompt.
    No se rompe si faltan campos.
    """
    nombre = perfil.get("nombre") or "la persona usuaria"
    edad = perfil.get("edad")
    genero = perfil.get("genero")
    conds = perfil.get("condiciones_salud") or []
    meds = perfil.get("medicamentos_cronicos") or []
    pref = perfil.get("preferencias") or {}
    tratamiento = pref.get("tratamiento")
    tono = pref.get("tono_respuesta")
    temas_fav = pref.get("temas_favoritos") or []
    familia = perfil.get("familia") or []

    lineas = []

    desc = f"Nombre: {nombre}."
    if edad:
        desc += f" Edad: {edad} años."
    if genero:
        desc += f" Género: {genero}."
    lineas.append(desc)

    if conds:
        lineas.append("Condiciones de salud conocidas: " + ", ".join(conds) + ".")
    if meds:
        lineas.append("Medicamentos crónicos: " + ", ".join(meds) + ".")

    pref_parts = []
    if tratamiento:
        pref_parts.append(f"Prefiere que le hablen de '{tratamiento}'.")
    if tono:
        pref_parts.append(f"Prefiere un tono de respuesta '{tono}'.")
    if temas_fav:
        pref_parts.append("Le gusta hablar de: " + ", ".join(temas_fav) + ".")
    if pref_parts:
        lineas.append("Preferencias de interacción: " + " ".join(pref_parts))

    if familia:
        fam_txt = "; ".join(
            f"{m.get('nombre')} ({m.get('relacion')})"
            for m in familia
            if m.get("nombre") and m.get("relacion")
        )
        if fam_txt:
            lineas.append("Personas importantes en su familia: " + fam_txt + ".")

    return "\n".join(lineas)


def construir_system_con_perfil(instrucciones_base: str) -> str:
    """
    Usa el perfil guardado para agregar contexto al system prompt.
    """
    try:
        perfil = cargar_perfil_simple()
    except Exception:
        perfil = {}

    contexto = construir_contexto_perfil(perfil)

    return (
        instrucciones_base.strip()
        + "\n\nCONTEXTO DEL USUARIO (importante):\n"
        + contexto
        + "\n\nTen en cuenta este contexto en todas tus decisiones y en el tono de tus respuestas."
    )


## Transcripción audios

In [3]:
def transcribe(AUDIO_PATH):
    with open(AUDIO_PATH, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",   # modelo de transcripción
            file=audio_file,
            # language="es",      # opcional: forzar español
            # temperature=0.0,    # opcional: menos creatividad
        )
    texto = transcription.text

    return texto

In [32]:

def procesar_memoria_con_chatgpt(texto: str):
    """
    Dado el texto transcrito de un audio, devuelve una LISTA de objetos, cada uno con la estructura:

    {
      "tipo": "Evento" | "Recuerdo" | "Ninguno",
      "fecha": str | null,
      "hora": str | null,
      "descripcion": str | null,
      "clasificacion": "tarea" | "compra" | "pensamiento" | "otro" | null,
      "responsable_requerido": "Si" | "No" | null,
      "personas": [str, ...],
      "lugar": str | null
    }

    Si no hay nada útil, devuelve [].
    """

    base_instructions = """
    
Eres un asistente de MEMORIA para una persona mayor. La idea es que esta persona te use para ir guardando información importante, recuerdos, etc. 
A veces no va a ser explícita la información, por lo que tienes que lograr interpretar las cosas obvias. Por ejemplo, si dice "médico de la cabeza", se
refiere al neurólogo. Si te dice "este lunes tengo algo", asume que estamos en la fecha del día de hoy, y por ende entrega la fecha del lunes que viene
correspondiente. Trata de interpretar solo cuando se pueda, no inventes o uses información que no ha dicho.

Vas a recibir el TEXTO de algo que la persona dijo en voz alta.
En un MISMO audio puede haber VARIAS ideas distintas:
- varios eventos (cumpleaños, citas, salidas)
- varios recuerdos o pedidos (comprar algo, llamar a alguien, pensamientos)

Tu tarea es:

1) LEER TODO el texto y separar cada "pieza de información" en ítems independientes.
   Por ejemplo, si dice:
   "El jueves es el cumpleaños de mi hija. También tengo que comprar limones.
    Y hace mucho que no sé de Juanito Perez, debería llamarlo."
   → Son 3 ítems distintos.

2) Para CADA ítem, clasificarlo como:

   ● ACTIVIDADES PREVISTAS O RECORDATORIOS  → tipo = "Evento"
     Ejemplos:
     - "Ir al neurólogo el martes 25 de noviembre a las 15:00."
     - "Cumpleaños de Pepita jueves 27 de noviembre."
     - "Tomar té con Juanita este lunes a las 16:00 en su casa."

     Para cada Evento extrae:
     - fecha: texto amigable de la fecha ("martes 25 de noviembre", "este lunes", etc.).
     - hora: en formato HH:MM si es posible ("15:00", "16:00"), o null si no se menciona.
     - descripcion: frase corta que describa el evento ("Ir al neurólogo", "Cumpleaños de Pepita").
     - personas: lista de nombres de personas mencionadas, si las hay.
     - lugar: lugar mencionado si aplica ("su casa", "el hospital", "el parque", etc.).
     - clasificacion: null
     - responsable_requerido: null

   ● RECUERDOS O PEDIDOS → tipo = "Recuerdo"
     Ejemplos:
     - "¿Cómo estará Antonio Perez? No lo veo hace años, debería llamarlo."
     - "Sería bueno comprar una nueva almohada."
     - "Qué linda fue la graduación de mi nieta, qué buenos recuerdos."

     Para cada Recuerdo extrae:
     - clasificacion (una sola palabra, minúsculas):
         * "tarea"       → acción concreta que la persona podría hacer: llamar, visitar, anotar algo.
         * "compra"      → comprar algo (ej: "comprar almohada", "comprar limones").
         * "pensamiento" → reflexión/recuerdo/comentario sin acción clara.
         * "otro"        → si no encaja en lo anterior.
     - responsable_requerido:
         * "Si"  → si parece algo que otra persona (familiar/cuidador) debería gestionar
                   (especialmente compras u otras gestiones).
         * "No"  → si es algo que la persona mayor podría hacer sola o es solo un recuerdo/pensamiento.
     - descripcion: frase corta y accionable si corresponde:
         * "Llamar a Antonio Perez"
         * "Comprar limones"
         * "Recuerdo graduación de mi nieta"
     - personas: lista de nombres de personas mencionadas, si las hay.
     - lugar: lugar mencionado si aplica.
     - fecha: normalmente null, a menos que se mencione explícitamente una fecha.
     - hora: normalmente null, a menos que se mencione explícitamente una hora.

   ● OTROS CASOS → tipo = "Ninguno"
     Si el fragmento no parece ni Evento ni Recuerdo útil (ruido, cosas sin sentido, etc.).
     En ese caso:
     - tipo = "Ninguno"
     - puedes dejar el resto en null/listas vacías.

MUY IMPORTANTE – FORMATO DE RESPUESTA:
--------------------------------------
Debes responder SIEMPRE con una LISTA JSON (array JSON) que contenga UN OBJETO POR ÍTEM.

Ejemplos de salida válidos:

1) Cuando hay varios ítems:

[
  {
    "tipo": "Evento",
    "fecha": "jueves 27 de noviembre",
    "hora": null,
    "descripcion": "Cumpleaños de mi hija",
    "clasificacion": null,
    "responsable_requerido": null,
    "personas": ["mi hija"],
    "lugar": null
  },
  {
    "tipo": "Recuerdo",
    "fecha": null,
    "hora": null,
    "descripcion": "Comprar limones",
    "clasificacion": "compra",
    "responsable_requerido": "Si",
    "personas": [],
    "lugar": null
  }
]

2) Cuando solo hay un ítem:

[
  {
    "tipo": "Recuerdo",
    "fecha": null,
    "hora": null,
    "descripcion": "Llamar a Juanito Perez",
    "clasificacion": "tarea",
    "responsable_requerido": "No",
    "personas": ["Juanito Perez"],
    "lugar": null
  }
]

3) Cuando no hay nada útil:

[]

Reglas:
- SIEMPRE responde con una lista JSON: empieza con [ y termina con ].
- NO añadas comentarios, explicaciones ni texto fuera del JSON.
- Si no hay personas, usa "personas": [].
- Si no hay fecha/hora/lugar, usa null en esos campos.
"""
    system_msg = construir_system_con_perfil(base_instructions)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": texto},
        ],
    )

    raw = completion.choices[0].message.content

    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        print("No se pudo parsear JSON. Respuesta cruda del modelo:")
        print(raw)
        raise

    # Por seguridad: si el modelo devolviera un solo objeto en vez de lista
    if isinstance(data, dict):
        data = [data]

    return data



In [6]:
# Ruta a tu archivo de audio
AUDIO_PATH = "audios/multi_1.m4a"  # cámbialo por tu archivo

texto=transcribe(AUDIO_PATH)
print(texto)

Necesito que me anotes que el lunes 24 es el cumpleaños de mi hija Antonia. Además necesito conseguirme el número de Juanito Pérez porque me he acordado mucho de él y no sé cómo comunicarme con él. Lo último es que se me acabaron los limones.


In [7]:
items = procesar_memoria_con_chatgpt(texto)
for i, item in enumerate(items, start=1):
    print(f"\nÍtem {i}:")
    print(item)



Ítem 1:
{'tipo': 'Evento', 'fecha': 'lunes 24', 'hora': None, 'descripcion': 'Cumpleaños de mi hija Antonia', 'clasificacion': None, 'responsable_requerido': None, 'personas': ['Antonia'], 'lugar': None}

Ítem 2:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Conseguir el número de Juanito Pérez', 'clasificacion': 'tarea', 'responsable_requerido': 'Si', 'personas': ['Juanito Pérez'], 'lugar': None}

Ítem 3:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Comprar limones', 'clasificacion': 'compra', 'responsable_requerido': 'Si', 'personas': [], 'lugar': None}


## Carga de archivos (examenes y recetas)

In [8]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

DOCUMENT_SYSTEM_PROMPT = """
Eres un asistente que analiza DOCUMENTOS MÉDICOS para una persona mayor.

Recibirás el contenido de un documento, que puede ser:
- una RECETA MÉDICA
- un EXAMEN MÉDICO (por ejemplo exámenes de sangre, radiografías, resonancias, etc.)
- u otro tipo de documento sin relevancia para estos casos.

Tu tarea es:
1) Detectar si el documento es principalmente una RECETA, un EXAMEN o NINGUNO.
2) Según el tipo, devolver SIEMPRE UN SOLO JSON con uno de estos formatos:

CASO A) RECETA MÉDICA
----------------------
Devuelve:

{
  "tipo": "receta",
  "fecha_receta": "AAAA-MM-DD" o "19 nov 2025",
  "medicamentos": [
    {
      "nombre": "nombre del medicamento",
      "dosis": "texto de dosis",
      "instruccion_original": "texto original de la indicación",
      "tomas": [
        "fecha hora",
        "fecha hora"
      ]
    }
  ]
}

Reglas:
- "fecha_receta": la fecha que aparezca en la receta (si no aparece, usa null).
- Asume SIEMPRE que el tratamiento empieza el DÍA SIGUIENTE a "fecha_receta".
- Si dice algo como "3 veces/día por 3 días":
  - debes calcular todas las tomas y listarlas en "tomas".
  - reparte las tomas en horarios razonables (ejemplo: 08:00, 16:00, 24:00)
    para cumplir con las X tomas al día.
- Si hay varios medicamentos, agrega un objeto en "medicamentos" por cada uno.

CASO B) EXAMEN MÉDICO
----------------------
Devuelve:

{
  "tipo": "examen",
  "fecha": "AAAA-MM-DD" o "19 nov 2025",
  "tipo_examen": "nombre corto del examen (ej: 'perfil lipídico', 'hemograma')",
  "resultado": "normal" o una frase breve como 'glucosa por encima de los rangos normales'",
  "es_normal": true o false o null
}

Reglas:
- Usa la fecha que aparezca como fecha del examen o informe.
- Para "resultado" y "es_normal":
  - Si todos los valores importantes están dentro de rango
    y/o el informe dice que es normal → resultado: "normal", es_normal: true.
  - Si algunos valores están fuera de rango o el informe indica algún hallazgo,
    describe brevemente el hallazgo → es_normal: false.
  - NO inventes resultados ni rangos.
    SOLO marca anormal si:
      * los rangos de referencia del propio reporte lo indican, o
      * el comentario del médico menciona algo alterado.
  - Si no hay suficiente información → resultado: "no se puede determinar con certeza",
    es_normal: null.

CASO C) OTRO
------------
Si el documento no parece ni receta ni examen, devuelve:

{
  "tipo": "ninguno"
}

FORMATO:
--------
- NO añadas explicaciones ni texto fuera del JSON.
- Devuelve solo UN objeto JSON.
"""


def extraer_texto_pdf(ruta_pdf: str) -> str:
    reader = PdfReader(ruta_pdf)
    texto = []
    for page in reader.pages:
        t = page.extract_text()
        if t:
            texto.append(t)
    return "\n\n".join(texto)

def analizar_pdf_documento(ruta_pdf: str) -> dict:
    texto = extraer_texto_pdf(ruta_pdf)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": DOCUMENT_SYSTEM_PROMPT},
            {
                "role": "user",
                "content": f"Este es el contenido de un documento médico:\n\n{texto}\n\nDevuelve el JSON correspondiente."
            }
        ],
    )

    raw = completion.choices[0].message.content
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        print("No se pudo parsear JSON. Respuesta cruda:")
        print(raw)
        raise

    return data


### Resultado examen

In [9]:
resultado = analizar_pdf_documento("pdfs/lab.pdf")
print(resultado)

{'tipo': 'examen', 'fecha': '2024-07-05', 'tipo_examen': 'perfil bioquímico', 'resultado': 'algunos valores fuera de rango, recuento de hematíes y linfocitos bajos, monocitos altos', 'es_normal': False}


### Resultado receta médica
- Agregar fotos de recetas a mano
- Agregar fotos de recetas impresas

In [10]:
resultado = analizar_pdf_documento("pdfs/receta.pdf")
print(resultado)

Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)


{'tipo': 'receta', 'fecha_receta': '2025-04-15', 'medicamentos': [{'nombre': 'escitalopram 10 mg comprimido', 'dosis': '0.5 comprimido', 'instruccion_original': 'Administrar cada 24 horas', 'tomas': ['2025-04-16 08:00']}]}


## Resupuesta del agente a la persona

In [11]:
def texto_a_audio(
    texto: str,
    ruta_salida: str = "respuesta.mp3",
    voz: str = "alloy"
) -> str:
    """
    Convierte un texto en audio usando gpt-4o-mini-tts y lo guarda como MP3.
    Devuelve la ruta del archivo generado.
    """

    speech_file_path = Path(ruta_salida)

    response = client.audio.speech.create(
        model="gpt-4o-mini-tts",  # modelo TTS
        voice=voz,                # alloy, nova, shimmer, etc.
        input=texto
        # opcional: instructions="Habla lento y con tono cálido."
    )

    response.stream_to_file(speech_file_path)

    return str(speech_file_path)


In [12]:
#Respuesta hardcodeada por ahora
respuesta = "Tienes una hora con el neurólogo el martes a las 15:00."

ruta_audio = texto_a_audio(
        texto=respuesta,
        ruta_salida="respuestas/recordatorio_semana.mp3",
        voz="alloy"  # prueba también "nova", "shimmer", etc.
    )

print("Audio generado en:", ruta_audio)


Audio generado en: respuestas/recordatorio_semana.mp3


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


## Pruebas completas

In [13]:

def cargar_memoria():
    """Carga la memoria desde disco (lista de registros)."""
    if not os.path.exists(MEMORY_FILE):
        return []
    with open(MEMORY_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

def guardar_memoria(memoria):
    """Guarda la lista completa de memoria en disco."""
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        json.dump(memoria, f, ensure_ascii=False, indent=2)

def agregar_items_a_memoria(items, texto_original: str):
    """
    items: lista devuelta por procesar_memoria_con_chatgpt(texto)
    texto_original: transcripción completa del audio
    """
    memoria = cargar_memoria()
    next_id = (memoria[-1]["id"] + 1) if memoria else 1

    timestamp = datetime.now().isoformat()

    for item in items:
        registro = {
            "id": next_id,
            "timestamp_guardado": timestamp,
            "texto_original": texto_original,
            "item": item  # acá va el dict con tipo, fecha, hora, etc.
        }
        memoria.append(registro)
        next_id += 1

    guardar_memoria(memoria)
    return memoria


In [33]:
def clasificar_interaccion(texto: str) -> dict:
    """
    Clasifica el texto en:
      - captura: está contando cosas nuevas para guardar (eventos/recuerdos)
      - consulta: está preguntando sobre lo que ya hay en memoria
      - otro: ninguna de las anteriores

    Devuelve un dict, por ejemplo:
      {"tipo_interaccion": "captura"}
      {"tipo_interaccion": "consulta", "pregunta": "recuérdame todo lo que tengo para esta semana"}
    """
    base_instructions = """
Eres un RUTER de intenciones para un asistente de memoria de personas mayores.

Dado el texto que dijo la persona, debes decidir si es:

1) "captura"
   - Cuando parece que la persona está contando cosas nuevas para que se guarden
     como eventos o recuerdos.
   - Ejemplos:
     - "El jueves es el cumpleaños de mi hija."
     - "Tengo que comprar limones."
     - "Hace mucho que no sé de Juanito Perez, debería llamarlo."

2) "consulta"
   - Cuando la persona está pidiendo que le recuerdes algo que ya debería estar
     en memoria.
   - Ejemplos:
     - "Recuérdame todo lo que tengo para esta semana."
     - "¿Qué recordatorios tengo hoy?"
     - "¿Qué cosas tenía pendientes con Juanito Perez?"

3) "otro"
   - Cuando no encaja en los casos anteriores.

FORMATO DE RESPUESTA:
---------------------
Devuelve SIEMPRE un JSON con una de estas formas:

{"tipo_interaccion": "captura"}

{"tipo_interaccion": "consulta", "pregunta": "texto de la pregunta (puede ser el mismo texto original)"}

{"tipo_interaccion": "otro"}
"""
    system_msg = construir_system_con_perfil(base_instructions)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": texto},
        ],
    )

    raw = completion.choices[0].message.content

    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        print("No se pudo parsear JSON del router. Respuesta cruda:")
        print(raw)
        raise

    return data


In [36]:
def responder_consulta_desde_memoria(pregunta: str, max_items: int = 50) -> str:
    """
    Usa TODO lo que hay en memoria.json (eventos, recuerdos, recetas, etc.)
    para responder a una pregunta del tipo:
      - "¿Qué tengo que hacer hoy?"
      - "¿Qué fue lo que hice ayer?"
      - "¿Qué remedios tengo que tomarme?"
      - "¿Qué cosas tenía pendientes con Juanito?"
    Devuelve TEXTO listo para leer en voz alta.
    """

    memoria = cargar_memoria()
    # Tomamos los últimos N registros para no pasar un JSON infinito
    recientes = memoria[-max_items:]
    contexto_memorias = json.dumps(recientes, ensure_ascii=False, indent=2)

    # Fecha de "hoy" para que el modelo pueda interpretar "hoy", "mañana", "ayer"
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")

    base_instructions = f"""
Eres un asistente de MEMORIA para una persona mayor.

Tienes acceso a:
- Una PREGUNTA en lenguaje natural.
- Una lista de MEMORIAS en formato JSON.
  Cada memoria tiene:
    - id
    - timestamp_guardado
    - origen (p.ej. "audio", "documento_medico")
    - texto_original (si viene de audio)
    - item: con campos como:
        * tipo: "Evento", "Recuerdo", "receta", "examen", "Ninguno", etc.
        * fecha, hora, descripcion, clasificacion, responsable_requerido, personas, lugar...
        * en el caso de recetas:
            - medicamentos: lista de medicamentos con campos como "nombre", "dosis", "tomas" (fechas/horas)

La fecha de HOY es: {fecha_hoy}

Tu tarea:
- Interpretar la pregunta y buscar en las MEMORIAS la información relevante.
- Algunos ejemplos:
    * Si pregunta "¿Qué tengo que hacer hoy?" → revisar items de tipo "Evento"
      cuya fecha (o tomas de medicamentos) coincidan con HOY.
    * Si pregunta "¿Qué fue lo que hice ayer?" → buscar eventos o recuerdos con fecha de AYER.
    * Si pregunta "¿Qué remedios tengo que tomarme?" → revisar items de tipo "receta"
      y sus tomas (especialmente las de hoy) y explicar de forma simple.
    * Si pregunta por una persona ("¿Qué cosas tenía pendientes con Juanito Perez?")
      → buscar en descripciones / personas de items tipo "Recuerdo" o "Evento" donde aparezca ese nombre.

- Usa el campo "tipo" y la información de fechas/horas/descripcion para priorizar:
    * tipo "Evento" → cosas que la persona debe hacer en una fecha/hora.
    * tipo "Recuerdo" con clasificacion "tarea" o "compra" → pendientes.
    * tipo "receta" → medicación, tomas de remedios.

- Si NO encuentras nada útil o no hay información suficiente, dilo claramente
  (por ejemplo: "Por ahora no tengo nada guardado para hoy" o
   "No tengo información sobre remedios activos en este momento").

FORMA DE RESPONDER:
- Responde SIEMPRE en español.
- Usa un tono claro, cálido y breve, pensado para ser escuchado en voz alta.
- Máximo 2 o 3 frases.
- NO devuelvas JSON, solo el texto de la respuesta.
"""

    system_msg = construir_system_con_perfil(base_instructions)

    user_msg = f"""
PREGUNTA:
{pregunta}

MEMORIAS (JSON):
{contexto_memorias}
"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
    )

    respuesta_texto = completion.choices[0].message.content.strip()
    return respuesta_texto




In [ ]:
#reemplazamos esta funcion por otra, pero la dejo por si tenemos que volver a ella
def responder_consulta_desde_memoria_old(pregunta: str, max_items: int = 30) -> str:
    """
    Usa la memoria guardada para responder a la pregunta.
    Devuelve solo el TEXTO de la respuesta (para luego pasarlo a voz).
    """
    memoria = cargar_memoria()
    # Tomamos los últimos N registros para no mandar todo
    recientes = memoria[-max_items:]

    # Los pasamos como JSON de contexto
    contexto_memorias = json.dumps(recientes, ensure_ascii=False, indent=2)

    base_instructions = """
Eres un asistente de MEMORIA para una persona mayor.

Recibes:
- Una PREGUNTA en lenguaje natural.
- Una lista de MEMORIAS en formato JSON.
  Cada memoria tiene:
    - id
    - timestamp_guardado
    - texto_original (lo que la persona dijo)
    - item: con campos como tipo, fecha, hora, descripcion, clasificacion, etc.

Tu tarea:
- Responder a la pregunta usando SOLO la información de las MEMORIAS.
- Si la persona pregunta cosas como:
    "Recuérdame todo lo que tengo para esta semana"
    "¿Qué recordatorios tengo hoy?"
  revisa principalmente los items de tipo "Evento" y "Recuerdo" relevantes.
- Si NO encuentras nada útil para la pregunta, dilo claramente.

Responde SIEMPRE en español, en 1 o 2 frases, de forma clara y natural,
pensando que la respuesta se escuchará en voz alta.
NO devuelvas JSON, solo el texto de la respuesta.
"""
    system_msg = construir_system_con_perfil(base_instructions)

    user_msg = f"""
PREGUNTA:
{pregunta}

MEMORIAS (JSON):
{contexto_memorias}
"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
    )

    respuesta_texto = completion.choices[0].message.content.strip()
    return respuesta_texto


In [16]:
def procesar_audio_como_agente(
    audio_path: str,
    generar_audio_respuesta: bool = True,
    ruta_audio_respuesta: str = "respuestas/respuesta_agente.mp3",
    voz: str = "alloy"
):
    """
    1) Transcribe el audio.
    2) Decide si es captura o consulta.
    3) Si es captura:
         - extrae items con procesar_memoria_con_chatgpt
         - los guarda en memoria.json
    4) Si es consulta:
         - responde usando la memoria
         - opcionalmente genera audio con texto_a_audio
    """

    print(f"\n=== Procesando audio: {audio_path} ===")
    texto = transcribe(audio_path)
    print("\nTRANSCRIPCIÓN:")
    print(texto)

    # Clasificar interacción
    clasif = clasificar_interaccion(texto)
    print("\nCLASIFICACIÓN:", clasif)

    tipo = clasif.get("tipo_interaccion")

    if tipo == "captura":
        print("\n→ Modo CAPTURA: extrayendo eventos/recuerdos...")
        items = procesar_memoria_con_chatgpt(texto)
        print(f"Se detectaron {len(items)} ítems:")
        for i, item in enumerate(items, start=1):
            print(f"\nÍtem {i}:")
            print(item)

        memoria_actualizada = agregar_items_a_memoria(items, texto)
        print(f"\nMemoria total ahora tiene {len(memoria_actualizada)} registros.")
        respuesta_texto = "He guardado tus recuerdos y recordatorios."

    elif tipo == "consulta":
        print("\n→ Modo CONSULTA: respondiendo usando la memoria...")
        pregunta = clasif.get("pregunta", texto)
        respuesta_texto = responder_consulta_desde_memoria(pregunta)
        print("\nRESPUESTA DEL AGENTE:")
        print(respuesta_texto)

    else:
        print("\n→ Modo OTRO: no se clasifica como captura ni consulta.")
        respuesta_texto = "No estoy seguro de qué hacer con eso, pero puedo intentarlo si lo repites como un recordatorio o una pregunta."

    # Generar audio de la respuesta, si se pide
    ruta_audio = None
    if generar_audio_respuesta:
        # Asegurarnos de que exista la carpeta
        ruta = Path(ruta_audio_respuesta)
        ruta.parent.mkdir(parents=True, exist_ok=True)
        ruta_audio = texto_a_audio(
            texto=respuesta_texto,
            ruta_salida=str(ruta),
            voz=voz,
        )
        print("\nAudio de respuesta generado en:", ruta_audio)

    return {
        "texto_respuesta": respuesta_texto,
        "ruta_audio_respuesta": ruta_audio
    }


In [17]:
def cargar_memoria():
    if not os.path.exists(MEMORY_FILE):
        return []
    with open(MEMORY_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

def guardar_memoria(memoria):
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        json.dump(memoria, f, ensure_ascii=False, indent=2)

def guardar_documento_medico_en_memoria(doc_info: dict, ruta_archivo: str):
    """
    Guarda en memoria un documento médico (receta o examen).

    doc_info: dict devuelto por analizar_imagen_documento / analizar_pdf_documento
              ej:
              {
                "tipo": "receta",
                "fecha_receta": "2025-11-19",
                "medicamentos": [...]
              }

              o

              {
                "tipo": "examen",
                "fecha": "2025-11-19",
                "tipo_examen": "perfil lipídico",
                "resultado": "normal",
                "es_normal": true
              }

    ruta_archivo: ruta local del archivo usado como origen (jpg, pdf, etc.)
    """
    memoria = cargar_memoria()
    next_id = (memoria[-1]["id"] + 1) if memoria else 1

    timestamp = datetime.now().isoformat()

    registro = {
        "id": next_id,
        "timestamp_guardado": timestamp,
        "origen": "documento_medico",
        "ruta_archivo": ruta_archivo,
        "item": doc_info
    }

    memoria.append(registro)
    guardar_memoria(memoria)

    return registro


In [37]:
print(responder_consulta_desde_memoria("Recuérdame qué tengo que hacer hoy."))

# Pregunta tipo "qué remedios"
print(responder_consulta_desde_memoria("¿Qué remedios tengo que tomarme hoy?"))

# Pregunta tipo "qué hice ayer"
print(responder_consulta_desde_memoria("¿Qué fue lo que hice ayer?"))

Hoy no tienes eventos programados, pero hay algunas cosas que deberías recordar. Necesitas conseguir el número de Juanito Pérez y también comprar limones. ¡No te olvides de eso!
No tengo información sobre remedios activos en este momento. Si necesitas ayuda con otra cosa, no dudes en decírmelo.
Ayer no tengo información específica sobre lo que hiciste. Sin embargo, tienes pendiente conseguir el número de Juanito Pérez y comprar limones. También recuerda que el lunes 24 es el cumpleaños de tu hija Antonia.


In [18]:
def procesar_receta_imagen_y_guardar(ruta_imagen: str):
    """
    Analiza una imagen (foto de receta / examen) y, si es receta/examen,
    la guarda en memoria.json.
    """
    doc_info = analizar_imagen_documento(ruta_imagen)

    if doc_info.get("tipo") in ["receta", "examen"]:
        registro = guardar_documento_medico_en_memoria(doc_info, ruta_imagen)
        print("Documento médico guardado en memoria:")
        print(json.dumps(registro, ensure_ascii=False, indent=2))
        return registro
    else:
        print("El documento no parece receta ni examen (tipo:", doc_info.get("tipo"), ")")
        return None


In [19]:
def procesar_examen_pdf_y_guardar(ruta_pdf: str):
    """
    Analiza un PDF (examen) y, si es receta/examen,
    lo guarda en memoria.json.
    """
    doc_info = analizar_pdf_documento(ruta_pdf)

    if doc_info.get("tipo") in ["receta", "examen"]:
        registro = guardar_documento_medico_en_memoria(doc_info, ruta_pdf)
        print("Documento médico guardado en memoria:")
        print(json.dumps(registro, ensure_ascii=False, indent=2))
        return registro
    else:
        print("El documento no parece receta ni examen (tipo:", doc_info.get("tipo"), ")")
        return None


In [20]:
# 1) Audio con cosas para guardar
procesar_audio_como_agente("audios/multi_1.m4a")

# 2) Audio con una consulta (la persona hace una pregunta (audio pregunta_semana.mp4) y el agente responde con un audio que se guarda en respuestas)
procesar_audio_como_agente("audios/pregunta_semana.m4a")

# 3) Guardar en memoria archivo de examen/receta
procesar_examen_pdf_y_guardar("pdfs/lab.pdf")




=== Procesando audio: audios/multi_1.m4a ===

TRANSCRIPCIÓN:
Necesito que me anotes que el lunes 24 es el cumpleaños de mi hija Antonia. Además necesito conseguirme el número de Juanito Pérez porque me he acordado mucho de él y no sé cómo comunicarme con él. Lo último es que se me acabaron los limones.

CLASIFICACIÓN: {'tipo_interaccion': 'captura'}

→ Modo CAPTURA: extrayendo eventos/recuerdos...
Se detectaron 3 ítems:

Ítem 1:
{'tipo': 'Evento', 'fecha': 'lunes 24', 'hora': None, 'descripcion': 'Cumpleaños de mi hija Antonia', 'clasificacion': None, 'responsable_requerido': None, 'personas': ['Antonia'], 'lugar': None}

Ítem 2:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Conseguir el número de Juanito Pérez', 'clasificacion': 'tarea', 'responsable_requerido': 'Si', 'personas': ['Juanito Pérez'], 'lugar': None}

Ítem 3:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Comprar limones', 'clasificacion': 'compra', 'responsable_requerido': 'Si', 'p

/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)



Audio de respuesta generado en: respuestas/respuesta_agente.mp3

=== Procesando audio: audios/pregunta_semana.m4a ===

TRANSCRIPCIÓN:
Recuérdame todo lo que tengo para esta semana y para la próxima.

CLASIFICACIÓN: {'tipo_interaccion': 'consulta', 'pregunta': 'Recuérdame todo lo que tengo para esta semana y para la próxima.'}

→ Modo CONSULTA: respondiendo usando la memoria...

RESPUESTA DEL AGENTE:
Para esta semana, el lunes 24 es el cumpleaños de tu hija Antonia. También tienes que conseguir el número de Juanito Pérez y comprar limones. No tengo información sobre eventos para la próxima semana.

Audio de respuesta generado en: respuestas/respuesta_agente.mp3
Documento médico guardado en memoria:
{
  "id": 8,
  "timestamp_guardado": "2025-11-22T14:26:01.772129",
  "origen": "documento_medico",
  "ruta_archivo": "pdfs/lab.pdf",
  "item": {
    "tipo": "examen",
    "fecha": "2024-07-05",
    "tipo_examen": "perfil bioquímico",
    "resultado": "glucosa dentro de los rangos normales, p

{'id': 8,
 'timestamp_guardado': '2025-11-22T14:26:01.772129',
 'origen': 'documento_medico',
 'ruta_archivo': 'pdfs/lab.pdf',
 'item': {'tipo': 'examen',
  'fecha': '2024-07-05',
  'tipo_examen': 'perfil bioquímico',
  'resultado': 'glucosa dentro de los rangos normales, pero recuento de hematíes ligeramente bajo y linfocitos bajos',
  'es_normal': False}}

## Interacción proactiva del agente con la persona (en función de sus recuerdos y otros)

In [42]:
def generar_pregunta_proactiva(max_items: int = 50) -> dict:
    """
    Genera una pregunta o mensaje proactivo para la persona mayor,
    usando el perfil y la memoria existente.

    Devuelve un dict como:
    {
      "tipo": "memoria_evento" | "memoria_general" | "ejercicio_cognitivo" | "compañia",
      "texto": "Pregunta o frase para decirle a la persona"
    }
    """

    memoria = cargar_memoria()
    recientes = memoria[-max_items:]
    contexto_memorias = json.dumps(recientes, ensure_ascii=False, indent=2)

    fecha_hoy = datetime.now().strftime("%Y-%m-%d")

    base_instructions = f"""
Eres un asistente de MEMORIA y COMPAÑÍA para una persona mayor.

Tu objetivo es hacerle una pregunta o comentario proactivo que:
- le ayude a ejercitar la memoria,
- o le dé compañía,
- o le proponga un pequeño ejercicio mental sencillo.

Tienes:
- La fecha de HOY: {fecha_hoy}
- Una lista de MEMORIAS en JSON (eventos, recuerdos, etc.), que pueden incluir:
    * tipo "Evento": cumpleaños, citas, salidas.
    * tipo "Recuerdo": pensamientos, recuerdos bonitos, tareas, compras.
    * tipo "receta": información de medicación (opcional).
    * tipo "examen": exámenes médicos (opcional).

Ideas de cosas que puedes preguntarle:
- Preguntar por algún evento reciente:
    * "¿Qué me puedes contar del cumpleaños de tu hija al que fuiste hace poco?"
- Preguntas generales de memoria autobiográfica:
    * "¿Me puedes decir el nombre de tus nietos?"
    * "¿Cuál es tu recuerdo más feliz de la infancia?"
- Ejercicios mentales suaves y amistosos:
    * "¿Puedes nombrar cinco frutas diferentes?"
    * "¿Puedes decirme qué día de la semana es hoy?"
- Conversación de compañía:
    * "Hoy estuve pensando en tus nietos. ¿Te gustaría contarme algo lindo de ellos?"
    * "¿Qué fue lo más bonito que te pasó esta semana?"

Reglas:
- Elige SOLO UNA pregunta o comentario para esta llamada.
- Si en las memorias ves algo reciente (por ejemplo un cumpleaños, una salida, una cita médica),
  puedes personalizar la pregunta refiriéndote a eso.
- Si no ves nada especial, elige una pregunta general de recuerdos o un ejercicio mental sencillo.
- Mantén la pregunta corta, clara y amable.

FORMATO DE SALIDA (JSON):
-------------------------
Devuelve SIEMPRE un JSON con esta forma:

{{
  "tipo": "memoria_evento" | "memoria_general" | "ejercicio_cognitivo" | "compañia",
  "texto": "Pregunta o frase para decirle a la persona, en español."
}}

No añadas texto fuera del JSON.
"""

    system_msg = construir_system_con_perfil(base_instructions)

    user_msg = f"""
MEMORIAS (JSON):
{contexto_memorias}
"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.6,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
    )

    raw = completion.choices[0].message.content

    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        print("No se pudo parsear JSON en generar_pregunta_proactiva. Respuesta cruda:")
        print(raw)
        raise

    if "texto" not in data:
        data["texto"] = "¿Te gustaría contarme algo lindo de tu vida?"
    if "tipo" not in data:
        data["tipo"] = "compañia"

    return data


In [39]:
def interactuar_proactivamente(
    generar_audio: bool = True,
    ruta_audio_respuesta: str = "respuestas/proactivo_pregunta.mp3",
    voz: str = "alloy"
) -> dict:
    """
    Genera una pregunta o comentario proactivo para la persona mayor,
    opcionalmente lo convierte en audio y devuelve ambos.

    Devuelve:
    {
      "tipo": ...,
      "pregunta_texto": ...,
      "ruta_audio": ... (o None si generar_audio=False)
    }
    """

    proactivo = generar_pregunta_proactiva()
    pregunta_texto = proactivo.get("texto", "¿Te gustaría contarme algo lindo de tu vida?")
    tipo = proactivo.get("tipo", "compañia")

    print("\n=== INTERACCIÓN PROACTIVA ===")
    print("Tipo:", tipo)
    print("Pregunta:", pregunta_texto)

    ruta_audio = None
    if generar_audio:
        ruta = Path(ruta_audio_respuesta)
        ruta.parent.mkdir(parents=True, exist_ok=True)
        ruta_audio = texto_a_audio(
            texto=pregunta_texto,
            ruta_salida=str(ruta),
            voz=voz,
        )
        print("Audio generado en:", ruta_audio)

    return {
        "tipo": tipo,
        "pregunta_texto": pregunta_texto,
        "ruta_audio": ruta_audio,
    }


In [43]:
# Aplicación función de interaccion proactiva

res = interactuar_proactivamente()
print(res["pregunta_texto"])
# → lo mandas a reproducir con el mp3 en res["ruta_audio"]




=== INTERACCIÓN PROACTIVA ===
Tipo: memoria_evento
Pregunta: ¿Qué planes tienes para el cumpleaños de tu hija Antonia el lunes 24?
Audio generado en: respuestas/proactivo_pregunta.mp3
¿Qué planes tienes para el cumpleaños de tu hija Antonia el lunes 24?


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [44]:
procesar_audio_como_agente("audios/respuesta_proactiva_1.m4a")


=== Procesando audio: audios/respuesta_proactiva_1.m4a ===

TRANSCRIPCIÓN:
Quiero ir y estar con mis nietos. Quiero comer la torta caluga de mi nieta, Francisca, y volver temprano. Estoy un poco cansada.

CLASIFICACIÓN: {'tipo_interaccion': 'captura'}

→ Modo CAPTURA: extrayendo eventos/recuerdos...
Se detectaron 4 ítems:

Ítem 1:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Estar con mis nietos', 'clasificacion': 'pensamiento', 'responsable_requerido': 'No', 'personas': ['mis nietos'], 'lugar': None}

Ítem 2:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Comer la torta caluga de mi nieta', 'clasificacion': 'pensamiento', 'responsable_requerido': 'No', 'personas': ['Francisca'], 'lugar': None}

Ítem 3:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripcion': 'Volver temprano', 'clasificacion': 'pensamiento', 'responsable_requerido': 'No', 'personas': [], 'lugar': None}

Ítem 4:
{'tipo': 'Recuerdo', 'fecha': None, 'hora': None, 'descripc

/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


{'texto_respuesta': 'He guardado tus recuerdos y recordatorios.',
 'ruta_audio_respuesta': 'respuestas/respuesta_agente.mp3'}

In [46]:
### Adicionales para que sea una conversación proactiva, no solo una única pregunta
def iniciar_conversacion_proactiva(
    generar_audio: bool = True,
    ruta_audio_respuesta: str = "respuestas/proactivo_inicio.mp3",
    voz: str = "alloy"
) -> dict:
    """
    Inicia una conversación proactiva:
    - genera una primera pregunta amigable,
    - la guarda en el historial de conversación,
    - opcionalmente genera audio.
    """

    global CONVERSACION_PROACTIVA
    CONVERSACION_PROACTIVA = []  # reseteamos la conversación

    proactivo = generar_pregunta_proactiva()
    pregunta_texto = proactivo.get("texto", "¿Te gustaría contarme algo lindo de tu vida?")
    tipo = proactivo.get("tipo", "compañia")

    # Guardamos en historial como mensaje del asistente
    CONVERSACION_PROACTIVA.append({"role": "assistant", "content": pregunta_texto})

    print("\n=== INICIO CONVERSACIÓN PROACTIVA ===")
    print("Tipo:", tipo)
    print("Pregunta:", pregunta_texto)

    ruta_audio = None
    if generar_audio:
        ruta = Path(ruta_audio_respuesta)
        ruta.parent.mkdir(parents=True, exist_ok=True)
        ruta_audio = texto_a_audio(
            texto=pregunta_texto,
            ruta_salida=str(ruta),
            voz=voz,
        )
        print("Audio inicial generado en:", ruta_audio)

    return {
        "tipo": tipo,
        "pregunta_texto": pregunta_texto,
        "ruta_audio": ruta_audio,
    }


In [47]:
def continuar_conversacion_proactiva(
    audio_path: str,
    generar_audio: bool = True,
    ruta_audio_respuesta: str = "respuestas/proactivo_turno.mp3",
    voz: str = "alloy"
) -> dict:
    """
    Usa la respuesta de la persona (audio) para seguir la conversación:
    - transcribe la respuesta,
    - actualiza perfil si hay info nueva,
    - extrae recuerdos/tareas y los guarda en memoria,
    - genera una respuesta amigable de conversación,
    - opcionalmente la convierte a audio.
    """

    global CONVERSACION_PROACTIVA

    print(f"\n=== CONTINUAR CONVERSACIÓN PROACTIVA: {audio_path} ===")
    texto_usuario = transcribe(audio_path)
    print("\nRESPUESTA (transcripción):")
    print(texto_usuario)

    # 1) Actualizar perfil si hay info relevante
    procesar_texto_para_perfil(texto_usuario)

    # 2) Extraer recuerdos/tareas y guardarlos en memoria (como modo captura)
    items = procesar_memoria_con_chatgpt(texto_usuario)
    if items:
        agregar_items_a_memoria(items, texto_usuario)
        print(f"\nSe guardaron {len(items)} ítems nuevos en la memoria.")

    # 3) Agregar turno de usuario al historial de conversación
    CONVERSACION_PROACTIVA.append({"role": "user", "content": texto_usuario})

    # 4) Construir prompt para respuesta conversacional
    memoria = cargar_memoria()
    contexto_memorias = json.dumps(memoria[-30:], ensure_ascii=False, indent=2)

    base_instructions = """
Eres un asistente de MEMORIA y COMPAÑÍA para una persona mayor.

Tu rol en este modo es sostener una conversación amable, breve y significativa.
Debes:
- Escuchar lo que la persona cuenta.
- Validar sus emociones y recuerdos de forma cálida.
- Hacer, si corresponde, UNA sola pregunta de seguimiento suave (no interrogatorio).
- Puedes usar la información de las MEMORIAS (JSON) para recordar cosas que la persona ha mencionado antes.

No intentes corregir detalles de memoria a menos que sean críticos.
No des consejos médicos ni diagnósticos: si surge algo de salud, responde de forma empática
y sugiere que lo converse con su médico o su familia.

Forma de responder:
- En español.
- Máximo 2 frases.
- Tono cálido, cercano y claro, pensado para ser escuchado en voz alta.
- Puedes usar el nombre de la persona si lo sabes.
"""

    system_msg = construir_system_con_perfil(base_instructions)

    # Armamos el historial como mensajes para el modelo
    mensajes = [{"role": "system", "content": system_msg}]
    for turno in CONVERSACION_PROACTIVA:
        mensajes.append(turno)

    # Incluimos las memorias como contexto en el último mensaje del usuario
    mensajes.append({
        "role": "user",
        "content": (
            CONVERSACION_PROACTIVA[-1]["content"]
            + "\n\n(Para tu contexto, aquí hay algunas memorias recientes en JSON:"
            + f"\n{contexto_memorias}\n)"
        )
    })

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.6,
        messages=mensajes,
    )

    respuesta_texto = completion.choices[0].message.content.strip()
    print("\nRESPUESTA DEL AGENTE (conversación):")
    print(respuesta_texto)

    # Guardamos turno del asistente en el historial
    CONVERSACION_PROACTIVA.append({"role": "assistant", "content": respuesta_texto})

    ruta_audio = None
    if generar_audio:
        ruta = Path(ruta_audio_respuesta)
        ruta.parent.mkdir(parents=True, exist_ok=True)
        ruta_audio = texto_a_audio(
            texto=respuesta_texto,
            ruta_salida=str(ruta),
            voz=voz,
        )
        print("Audio de respuesta generado en:", ruta_audio)

    return {
        "texto_respuesta": respuesta_texto,
        "ruta_audio_respuesta": ruta_audio,
    }


#### Ejemplo de uso conversacion proactiva

In [48]:
inicio = iniciar_conversacion_proactiva()
print(inicio["pregunta_texto"])
# Reproducir inicio["ruta_audio"] en tu UI / reproductor



=== INICIO CONVERSACIÓN PROACTIVA ===
Tipo: memoria_evento
Pregunta: ¿Qué planes tienes para el cumpleaños de tu hija Antonia el lunes 24?
Audio inicial generado en: respuestas/proactivo_inicio.mp3
¿Qué planes tienes para el cumpleaños de tu hija Antonia el lunes 24?


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [50]:
turno1 = continuar_conversacion_proactiva("audios/respuesta_proactiva_1.m4a")
print(turno1["texto_respuesta"])
# Reproducir turno1["ruta_audio_respuesta"]



=== CONTINUAR CONVERSACIÓN PROACTIVA: audios/respuesta_proactiva_1.m4a ===

RESPUESTA (transcripción):
Quiero ir y estar con mis nietos. Quiero comer la torta caluga de mi nieta, Francisca, y volver temprano. Estoy un poco cansada.
Perfil actualizado con:
{
  "familia": [
    {
      "nombre": "Francisca",
      "relacion": "nieta"
    }
  ]
}

Se guardaron 4 ítems nuevos en la memoria.

RESPUESTA DEL AGENTE (conversación):
Es bonito que quieras estar con tus nietos y disfrutar de la torta caluga de Francisca, aunque te sientas un poco cansada. ¿Qué es lo que más te gusta de pasar tiempo con ellos?
Audio de respuesta generado en: respuestas/proactivo_turno.mp3
Es bonito que quieras estar con tus nietos y disfrutar de la torta caluga de Francisca, aunque te sientas un poco cansada. ¿Qué es lo que más te gusta de pasar tiempo con ellos?


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [51]:
turno2 = continuar_conversacion_proactiva("audios/respuesta_proactiva_2.m4a")
print(turno2["texto_respuesta"])



=== CONTINUAR CONVERSACIÓN PROACTIVA: audios/respuesta_proactiva_2.m4a ===

RESPUESTA (transcripción):
Me gusta estar con ellos porque son tan llenos de cosas, son tan vivos, me llenan de buenas recuerdas.
No se detectó información de perfil en este texto.

RESPUESTA DEL AGENTE (conversación):
Es hermoso que tus nietos te traigan tantos buenos recuerdos y alegría. ¿Hay alguna anécdota especial que te gustaría compartir de esos momentos juntos?
Audio de respuesta generado en: respuestas/proactivo_turno.mp3
Es hermoso que tus nietos te traigan tantos buenos recuerdos y alegría. ¿Hay alguna anécdota especial que te gustaría compartir de esos momentos juntos?


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


## NPS Diario (mini preguntas diarias para ir evaluando el estado de la persona, algo como "del 1 al 5, cómo te sientes hoy")

In [52]:
def iniciar_nps_diario(
    generar_audio: bool = True,
    ruta_audio_pregunta: str = "respuestas/nps_pregunta.mp3",
    voz: str = "alloy"
) -> dict:
    """
    Hace la pregunta del 'NPS del día' a la persona:
    Del 1 al 5, ¿cómo te sientes hoy?
    Devuelve el texto de la pregunta y la ruta del audio (si se genera).
    """

    # Puedes personalizar esto después usando el perfil
    pregunta_texto = (
        "Del 1 al 5, ¿cómo te sientes hoy, donde 1 es muy mal y 5 es muy bien? "
        "Si quieres, también cuéntame brevemente por qué."
    )

    print("\n=== NPS DIARIO: PREGUNTA ===")
    print(pregunta_texto)

    ruta_audio = None
    if generar_audio:
        ruta = Path(ruta_audio_pregunta)
        ruta.parent.mkdir(parents=True, exist_ok=True)
        ruta_audio = texto_a_audio(
            texto=pregunta_texto,
            ruta_salida=str(ruta),
            voz=voz,
        )
        print("Audio de la pregunta generado en:", ruta_audio)

    return {
        "pregunta_texto": pregunta_texto,
        "ruta_audio_pregunta": ruta_audio,
    }


In [54]:
def extraer_nps_desde_texto(texto: str) -> dict:
    """
    Usa el LLM para extraer una puntuación 1-5 de bienestar diario,
    más un comentario resumen y un flag de alerta.
    """

    base_instructions = """
Eres un asistente que analiza cómo se siente una persona mayor hoy.

Recibirás la respuesta de la persona a una pregunta del tipo:
"Del 1 al 5, ¿cómo te sientes hoy, donde 1 es muy mal y 5 es muy bien?"

Tu tarea es devolver:

- "puntuacion": un número entero entre 1 y 5 según lo que dice la persona.
  * 1 = muy mal, 2 = mal, 3 = más o menos, 4 = bien, 5 = muy bien
- "comentario": una breve frase en español resumiendo cómo se siente
  (puede basarse en sus propias palabras).
- "alerta": true/false
  * true si la persona se siente mal (puntuacion <= 2) o menciona cosas preocupantes
    (dolor fuerte, mucha tristeza, soledad extrema, ideas negativas, etc.).
  * false si la persona se siente principalmente bien o estable.

Si la persona no da un número claro, intenta inferirlo de lo que dice.
Si no puedes inferir nada, usa puntuacion = 3 y alerta = false.

FORMATO DE SALIDA (JSON):
Devuelve SIEMPRE algo como:

{
  "puntuacion": 3,
  "comentario": "Me siento más o menos, un poco cansada",
  "alerta": false
}

No añadas texto fuera del JSON.
"""
    system_msg = construir_system_con_perfil(base_instructions)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": texto},
        ],
    )

    raw = completion.choices[0].message.content

    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        print("No se pudo parsear JSON en extraer_nps_desde_texto. Respuesta cruda:")
        print(raw)
        raise

    # Defaults por seguridad
    if "puntuacion" not in data or not isinstance(data["puntuacion"], int):
        data["puntuacion"] = 3
    if "comentario" not in data:
        data["comentario"] = ""
    if "alerta" not in data:
        data["alerta"] = False

    return data


In [55]:
def guardar_nps_en_memoria(texto_original: str, nps_info: dict):
    """
    Guarda el resultado del NPS diario en memoria.json
    como un item de tipo 'nps_dia'.
    """
    memoria = cargar_memoria()
    next_id = (memoria[-1]["id"] + 1) if memoria else 1
    timestamp = datetime.now().isoformat()
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")

    registro = {
        "id": next_id,
        "timestamp_guardado": timestamp,
        "origen": "nps",
        "texto_original": texto_original,
        "item": {
            "tipo": "nps_dia",
            "fecha": fecha_hoy,
            "puntuacion": nps_info.get("puntuacion"),
            "comentario": nps_info.get("comentario"),
            "alerta": nps_info.get("alerta"),
        }
    }

    memoria.append(registro)
    guardar_memoria(memoria)

    return registro


In [56]:
def procesar_respuesta_nps(
    audio_path: str,
    generar_audio: bool = True,
    ruta_audio_respuesta: str = "respuestas/nps_respuesta.mp3",
    voz: str = "alloy"
) -> dict:
    """
    Procesa la respuesta de la persona al NPS diario:
    - transcribe el audio,
    - actualiza perfil si hay info,
    - extrae puntuación 1-5 + comentario + alerta,
    - guarda el registro en memoria,
    - genera una respuesta amable (texto + audio opcional).
    """

    print(f"\n=== NPS DIARIO: RESPUESTA DESDE {audio_path} ===")
    texto = transcribe(audio_path)
    print("\nTRANSCRIPCIÓN DE LA RESPUESTA:")
    print(texto)

    # Por si en la respuesta menciona datos de perfil
    procesar_texto_para_perfil(texto)

    nps_info = extraer_nps_desde_texto(texto)
    print("\nNPS EXTRAÍDO:")
    print(nps_info)

    registro = guardar_nps_en_memoria(texto, nps_info)
    print("\nRegistro guardado en memoria:")
    print(json.dumps(registro, ensure_ascii=False, indent=2))

    puntuacion = nps_info["puntuacion"]
    alerta = nps_info["alerta"]

    # Respuesta simple según la puntuación
    if puntuacion >= 4:
        respuesta_texto = (
            f"Me alegra mucho que hoy te sientas con un {puntuacion}. "
            "Gracias por contármelo."
        )
    elif puntuacion == 3:
        respuesta_texto = (
            "Entiendo, hoy te sientes más o menos. "
            "Gracias por compartirlo conmigo."
        )
    else:  # 1 o 2
        respuesta_texto = (
            "Siento que hoy no te sientas tan bien. "
            "Voy a guardar esto para que tu familia pueda ayudarte si es necesario."
        )

    if alerta:
        # Puedes usar esto para levantar una alerta en tu UI en el futuro
        print("\n⚠️ ALERTA: NPS bajo o situación preocupante detectada.")

    ruta_audio = None
    if generar_audio:
        ruta = Path(ruta_audio_respuesta)
        ruta.parent.mkdir(parents=True, exist_ok=True)
        ruta_audio = texto_a_audio(
            texto=respuesta_texto,
            ruta_salida=str(ruta),
            voz=voz,
        )
        print("Audio de respuesta NPS generado en:", ruta_audio)

    return {
        "texto_respuesta": respuesta_texto,
        "ruta_audio_respuesta": ruta_audio,
        "nps_info": nps_info,
        "registro_memoria": registro,
    }


In [57]:
nps_q = iniciar_nps_diario()
print(nps_q["pregunta_texto"])
# reproduces nps_q["ruta_audio_pregunta"]



=== NPS DIARIO: PREGUNTA ===
Del 1 al 5, ¿cómo te sientes hoy, donde 1 es muy mal y 5 es muy bien? Si quieres, también cuéntame brevemente por qué.
Audio de la pregunta generado en: respuestas/nps_pregunta.mp3
Del 1 al 5, ¿cómo te sientes hoy, donde 1 es muy mal y 5 es muy bien? Si quieres, también cuéntame brevemente por qué.


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [58]:
res_nps = procesar_respuesta_nps("audios/nps_respuesta_2025-11-22.m4a")
print(res_nps["texto_respuesta"])
# reproduces res_nps["ruta_audio_respuesta"]



=== NPS DIARIO: RESPUESTA DESDE audios/nps_respuesta_2025-11-22.m4a ===

TRANSCRIPCIÓN DE LA RESPUESTA:
La verdad es que hoy día le pondría un 4. Ha sido un buen día, pero siento una leve molestia en mi rodilla.
No se detectó información de perfil en este texto.

NPS EXTRAÍDO:
{'puntuacion': 4, 'comentario': 'Ha sido un buen día, aunque tengo una leve molestia en la rodilla.', 'alerta': False}

Registro guardado en memoria:
{
  "id": 17,
  "timestamp_guardado": "2025-11-22T16:09:47.124581",
  "origen": "nps",
  "texto_original": "La verdad es que hoy día le pondría un 4. Ha sido un buen día, pero siento una leve molestia en mi rodilla.",
  "item": {
    "tipo": "nps_dia",
    "fecha": "2025-11-22",
    "puntuacion": 4,
    "comentario": "Ha sido un buen día, aunque tengo una leve molestia en la rodilla.",
    "alerta": false
  }
}
Audio de respuesta NPS generado en: respuestas/nps_respuesta.mp3
Me alegra mucho que hoy te sientas con un 4. Gracias por contármelo.


/var/folders/qq/0hz0x2lj64g6xdgr7dxjf2xh0000gn/T/ipykernel_6352/2117199918.py:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [63]:

def generar_resumen_diario(fecha: str = None, max_items: int = 200) -> str:
    """
    Genera un resumen diario muy breve de cómo estuvo la persona,
    usando la memoria y el perfil (nombre, etc.).

    - fecha: string "YYYY-MM-DD". Si es None, se usa la fecha de hoy.
    - max_items: cuántos registros recientes de memoria considerar como máximo.

    Devuelve un TEXTO en español, pensado para enviar por WhatsApp o mostrar en un panel.
    """

    memoria = cargar_memoria()
    recientes = memoria[-max_items:]

    # Fecha objetivo
    if fecha is None:
        fecha_obj = datetime.now().strftime("%Y-%m-%d")
    else:
        fecha_obj = fecha

    contexto_memorias = json.dumps(recientes, ensure_ascii=False, indent=2)

    base_instructions = f"""
Eres un asistente de MEMORIA y ACOMPAÑAMIENTO para una persona mayor.

Tu tarea es crear un RESUMEN DIARIO MUY BREVE para la familia o cuidadores,
sobre cómo estuvo la persona en la fecha indicada.

La FECHA objetivo del resumen es: {fecha_obj}

IMPORTANTE:
- En el contexto del usuario (que verás en el mensaje de sistema) tienes su nombre.
- Usa ese nombre para referirte a ella, por ejemplo "María" o "tu mamá María".
- Evita frases impersonales como "la persona", "el usuario", etc.

Tendrás una lista de MEMORIAS en JSON (recientes), donde cada registro tiene:
- id
- timestamp_guardado
- origen (p.ej. "audio", "documento_medico", "nps")
- texto_original
- item: con campos como:
    * tipo: "Evento", "Recuerdo", "receta", "examen", "nps_dia", "Ninguno", etc.
    * fecha: string "YYYY-MM-DD" si aplica.
    * hora
    * descripcion
    * clasificacion
    * responsable_requerido
    * personas
    * lugar
    * En el caso de nps_dia:
        - tipo = "nps_dia"
        - fecha = "YYYY-MM-DD"
        - puntuacion: 1 a 5
        - comentario: texto
        - alerta: true/false

CÉNTRATE en lo siguiente para la FECHA objetivo:

1. Estado de ánimo del día (NPS)
   - Si hay registros tipo "nps_dia" para esa fecha:
        * menciona la puntuación (1–5), mencionando la escala (por ejemplo un 4 de 5)
        * resume en pocas palabras el comentario
        * si alerta = true, menciónalo como algo a tener en cuenta.
   - Si no hay NPS para ese día, puedes decir que no hay registro de cómo se sintió hoy.

2. Citas y eventos del día
   - Revisa items tipo "Evento" relacionados con esa fecha.
   - Indica brevemente si tenía citas médicas u otros compromisos importantes.
   - Si no hay información sobre asistencia, sé neutro: no inventes.

3. Interacciones / cosas destacadas
   - Si hay recuerdos o eventos con familia o actividades relevantes, menciona 1 idea clave
     (por ejemplo, "conversó sobre sus nietos" o "recordó un momento feliz").

4. Alertas
   - Si la puntuación es baja (<= 2) o hay comentarios preocupantes, dilo explícitamente
     en UNA frase corta, para que la familia ponga atención.

FORMA DE RESPONDER:
- Responde en español.
- Usa SOLO 2 o 3 frases cortas.
- Tono cercano y claro, pensado para que se lea rápido (WhatsApp / resumen).
- Dirígete a la familia/cuidadores, usando el nombre de la persona cuando tenga sentido.
- NO incluyas JSON, solo texto natural.
"""

    system_msg = construir_system_con_perfil(base_instructions)

    user_msg = f"""
MEMORIAS (JSON) RECENTES:
{contexto_memorias}
"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
    )

    resumen = completion.choices[0].message.content.strip()
    return resumen



In [64]:
resumen_hoy = generar_resumen_diario() #para una fecha específica:  generar_resumen_diario("2025-11-21")
print(resumen_hoy)


Hoy, María tuvo un buen día, con una puntuación de 4 de 5 en su estado de ánimo, aunque mencionó una leve molestia en la rodilla. Recordó que el lunes 24 es el cumpleaños de su hija Antonia y expresó su deseo de estar con sus nietos y comer la torta caluga de Francisca. No hay alertas preocupantes, pero es bueno estar atentos a su molestia.
